# ロト特徴量システム × PostgreSQL 連携レポート

このノートブックでは、`loto_feature_system_v2` プロジェクトのロト抽選データを PostgreSQL に取り込み、
テーブル内容を確認するまでの一連の手順をまとめます。

## 想定ディレクトリ構成（例）

```text
C:/nf/
 ├─ loto_feature_system_v2/
 │   ├─ db/
 │   │   ├─ db_config.py
 │   │   ├─ postgres_manager.py
 │   │   ├─ setup_postgres.py
 │   │   ├─ loto_etl.py
 │   │   └─ loto_pg_store.py
 └─ （その他のプロジェクト）
```

以降では **Windows + PostgreSQL + Python 3.11** を想定しています。
パスは自分の環境に合わせて適宜書き換えてください。

## 0. 事前準備

1. PostgreSQL のインストール・起動  
2. `loto_feature_system_v2` リポジトリの配置（例: `C:/nf/loto_feature_system_v2`）  
3. Python 仮想環境の作成と必要ライブラリのインストール

### 0-1. 仮想環境の例（任意）

PowerShell などのシェルで:

```powershell
cd C:/nf/loto_feature_system_v2
python -m venv .venv
.venv\Scripts\activate

pip install -r requirements.txt
# PostgreSQL 接続用
pip install psycopg2-binary pandas
```

### 0-2. `db_config.py` の確認

`db/db_config.py` 内の `DB_CONFIG` を、自分の PostgreSQL 環境に合わせて修正します。

```python
DB_CONFIG = {
    "host": "127.0.0.1",
    "port": 5432,
    "database": "postgres",
    "user": "postgres",
    "password": "（自分のパスワード）",
}
```

ここで設定した情報を、このノートブックからもそのまま利用します。

## 1. 作業ディレクトリの移動

まず、`db` ディレクトリを作業ディレクトリにします。
環境に合わせてパスを変更してください。

In [31]:
# ★ パスは自分の環境に合わせて書き換えてください
# 例: C:/nf/loto_feature_system_v2/db
project_db_dir = r"C:/nf/loto_feature_system_v2/db"

import os
os.chdir(project_db_dir)
print("現在のディレクトリ:", os.getcwd())

現在のディレクトリ: C:\nf\loto_feature_system_v2\db


## 2. PostgreSQL への接続確認 (`setup_postgres.py test`)

`setup_postgres.py` の `test` サブコマンドを使って、
`db_config.py` の設定で PostgreSQL に接続できるかを確認します。

In [32]:
# PostgreSQL への接続テスト
# db_config.py の DB_CONFIG を修正したあとで実行してください
!python setup_postgres.py test


PostgreSQL 接続テスト

接続情報:
  Host: 127.0.0.1
  Port: 5432
  Database: postgres
  User: postgres
✓ PostgreSQL接続成功: 127.0.0.1:5432/postgres

✓ 接続成功

PostgreSQL バージョン:
  PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35217, 64-bit

✓ PostgreSQL接続を閉じました


## 3. `nf_` テーブルの一覧・構造確認 (`setup_postgres.py info`)

既に存在する `nf_` プレフィックス付きテーブルの一覧や、
各テーブルのカラム情報・レコード数を確認することができます。

In [33]:
# nf_ テーブルの一覧とカラム情報、レコード数を確認
!python setup_postgres.py info


PostgreSQL テーブル情報
✓ PostgreSQL接続成功: 127.0.0.1:5432/postgres

nf_*テーブル: 28 個

テーブル: nf_calibration
--------------------------------------------------------------------------------
column_name        data_type max_length nullable
     cal_id           bigint       None       NO
     run_id           bigint       None      YES
   model_id           bigint       None      YES
      level          integer       None      YES
   coverage double precision       None      YES
        mil double precision       None      YES
    winkler double precision       None      YES

レコード数: 0

テーブル: nf_ckpt
--------------------------------------------------------------------------------
                                                    column_name                   data_type max_length nullable
                                                             id                     integer       None       NO
dir_data_long_parent_dir_n_features_loto_bingo5_model_autokan_n                        text       

## 4. 既存 `nf_` テーブルの削除とクリーンセットアップ (`setup_postgres.py setup`)

過去の実験で作成した `nf_` テーブルを一度リセットしたい場合は、
`setup_postgres.py setup` を実行します。

実行中に `削除しますか？ (y/N)` のような確認プロンプトが表示されるので、
本当に削除して良い場合だけ `y` を入力してください。

## 5. ロト CSV の取得と `nf_loto_final` テーブルへの保存 (`loto_pg_store.py`)

`loto_pg_store.py` は以下の処理をまとめて行います。

1. `loto_etl.py` を使って、ロト関連の CSV をすべて取得  
2. CSV を縦長の時系列 DataFrame に整形 (`loto`, `num`, `ds`, `unique_id`, `y`, `CO` など)  
3. PostgreSQL に `nf_loto_final` テーブルを作成  
4. 主キー `(loto, num, unique_id)` で UPSERT（既存行は更新、それ以外は挿入）  

作成されるテーブルの主なカラムは次のとおりです。

- `loto` : シリーズ名（`mini`, `loto6`, `loto7`, `bingo5`, `num3`, `num4` など）  
- `num` : 開催回（整数）  
- `ds` : 抽選日 (`TIMESTAMP`)  
- `unique_id` : `N1` 〜 `N7` などの番号スロット名  
- `y` : 抽選数字  
- `CO` : キャリーオーバー金額  
- `N1NU` / `N1PM` / … : 各等級の口数・賞金など

In [ ]:
# ロトの全 CSV を取得し、nf_loto_final テーブルへ一括 UPSERT します
!python loto_pg_store.py

UPSERT行数: 75202


## 6. Python + pandas で `nf_loto_final` テーブルを参照

ここからは、`psycopg2` と `pandas` を使って、
`nf_loto_final` テーブルの中身を直接確認してみます。

In [ ]:
import psycopg2
import pandas as pd

from db_config import DB_CONFIG, TABLE_PREFIX

# PostgreSQL に接続
conn = psycopg2.connect(**DB_CONFIG)
table_name = f"{TABLE_PREFIX}loto_final"
print("接続完了:", DB_CONFIG["host"], DB_CONFIG["database"], "テーブル:", table_name)

接続完了: 127.0.0.1 postgres テーブル: nf_loto_final


In [36]:
# 6-1. 先頭 20 行を確認
query_head = f"""
SELECT *
FROM {table_name}
ORDER BY ds, loto, num, unique_id;
"""

df_head = pd.read_sql(query_head, conn)
df_head

C:\Users\hashimoto.ryohei\AppData\Local\Temp\ipykernel_24680\2935150515.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_head = pd.read_sql(query_head, conn)


InterfaceError: connection already closed

In [35]:
df_head.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   loto       20 non-null     object        
 1   num        20 non-null     int64         
 2   ds         20 non-null     datetime64[ns]
 3   unique_id  20 non-null     object        
 4   y          20 non-null     int64         
 5   co         20 non-null     int64         
 6   n1nu       20 non-null     float64       
 7   n1pm       20 non-null     float64       
 8   n2nu       20 non-null     float64       
 9   n2pm       20 non-null     float64       
 10  n3nu       20 non-null     float64       
 11  n3pm       20 non-null     float64       
 12  n4nu       20 non-null     float64       
 13  n4pm       20 non-null     float64       
 14  n5nu       20 non-null     float64       
 15  n5pm       20 non-null     float64       
 16  n6nu       20 non-null     float64       
 17 

In [ ]:
# 6-2. シリーズごとの行数を確認
query_count = f"""
SELECT loto, COUNT(*) AS n
FROM {table_name}
GROUP BY loto
ORDER BY loto;
"""

df_count = pd.read_sql(query_count, conn)
df_count

C:\Users\hashimoto.ryohei\AppData\Local\Temp\ipykernel_24680\3655072092.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_count = pd.read_sql(query_count, conn)


,loto,n
0,bingo5,3560
1,loto6,12300
2,loto7,4557
3,mini,6800
4,num3,20565
5,num4,27420


In [ ]:
df_count.loto.tolist()  

['bingo5', 'loto6', 'loto7', 'mini', 'num3', 'num4']

## 7. サンプルクエリ

### 7-1. 特定シリーズ・期間のデータを取得

例として、`loto6` の 2020 年分のデータを抽出してみます。

In [ ]:
# 7-1. 特定シリーズ・期間の抽出例
query_range = f"""
SELECT *
FROM {table_name}
WHERE loto = 'loto6'
  AND ds BETWEEN '2020-01-01' AND '2020-12-31'
ORDER BY ds, num, unique_id
LIMIT 100;
"""

df_range = pd.read_sql(query_range, conn)
df_range

C:\Users\hashimoto.ryohei\AppData\Local\Temp\ipykernel_24680\1340427911.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_range = pd.read_sql(query_range, conn)


InterfaceError: connection already closed

### 7-2. ある回の当選数字一覧を取得

`loto_pg_store.py` に用意されているヘルパー関数
`read_by_loto_num(loto, num)` と同じイメージのクエリを、
SQL で直接実行してみます。

In [ ]:
['bingo5', 'loto6', 'loto7', 'mini', 'num3', 'num4']
# 7-2. ある回の当選数字一覧（例: loto6, 第 1500 回）
target_loto = "num3"
target_num = 1500

query_one_draw = f"""
SELECT *
FROM {table_name}
WHERE loto = %s
  AND num = %s
ORDER BY unique_id;
"""

df_one_draw = pd.read_sql(query_one_draw, conn, params=[target_loto, target_num])
df_one_draw

C:\Users\hashimoto.ryohei\AppData\Local\Temp\ipykernel_24680\2377180805.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_one_draw = pd.read_sql(query_one_draw, conn, params=[target_loto, target_num])


,loto,num,ds,unique_id,y,co,n1nu,n1pm,n2nu,n2pm,n3nu,n3pm,n4nu,n4pm,n5nu,n5pm,n6nu,n6pm,n7nu,n7pm
0,num3,1500,2005-02-14,N1,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,num3,1500,2005-02-14,N2,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,num3,1500,2005-02-14,N3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 8. 接続のクローズ

最後に、開いた PostgreSQL 接続を明示的に閉じておきます。

In [ ]:
conn.close()
print("PostgreSQL 接続をクローズしました。")

PostgreSQL 接続をクローズしました。
